In [ ]:
# Mutations_analysis by Marco Fumasoni

# This code has been written to analyze the list of mutations obtained from mutantanalysis by John Koschwanez (https://github.com/koschwanez/mutantanalysis)
# The script identify putative adaptive mutations when a gene is mutated more frequently than expected by chance across parallel
# and independent populations. The script run the same analysis in a list of mutations obtained from evolved wt, to remove from the 
# candidate adaptive genes, the one that are likely due to experimental conditions


# NOTE
# The code is made available for transparency reasons. At present, it is not intended to be readily usable on different datasets. 
# Also, it was not annotated and compiled to be user-friendly. Please, contact me privately for any inquiry related to the code usage.
# I will maintain this code with improved versions as soon as they are developed.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
import xlsxwriter
import csv

In [3]:
sns.set_context('paper', font_scale=2.2)

In [4]:
sns.set_style('ticks')

In [5]:
import pandas as pd

In [6]:
from collections import Counter

In [7]:
#Import the mutations find in the ancestor strains compared to a wt
AncVar = pd.read_csv('tabbed_output_by_gene_anc.txt', sep='\t')

#Import the mutations found at 10% in the wt evolved lines (populations - 1000 generations)
ListByGenes1 = pd.read_csv('tabbed_output_by_gene_100_10.txt', sep='\t')
ListByGenes2 = pd.read_csv('tabbed_output_by_gene_108_10.txt', sep='\t')

#Import the mutations found at 10% in the ctf4 haploid evolved lines (populations - 1000 generations)
ListByGenes3 = pd.read_csv('tabbed_output_by_gene_yMF117_10.txt', sep='\t')
ListByGenes4 = pd.read_csv('tabbed_output_by_gene_yMF124_10.txt', sep='\t')

#Import the mutations found at 75% in the ctf4 haploid evolved lines (Clones - 1000 generations)
ListByGenes5 = pd.read_csv('tabbed_output_by_gene_117.txt', sep='\t')
ListByGenes6 = pd.read_csv('tabbed_output_by_gene_124.txt', sep='\t')

# Create separate lists for wt, ctf4 (pop and colones)

frames = [ListByGenes1, ListByGenes2]
wt_evo = pd.concat(frames, ignore_index=True)

frames = [ListByGenes3, ListByGenes4]
ctf4_pop = pd.concat(frames, ignore_index=True)

frames = [ListByGenes5, ListByGenes6]
ctf4_clones = pd.concat(frames, ignore_index=True)

In [8]:
# Consider only the fwd mutations for both evolved wt and evolved mut
wt_evo = wt_evo.loc[wt_evo['mutation_direction'] == 'forward mutation', :]
ctf4_pop = ctf4_pop.loc[ctf4_pop['mutation_direction'] == 'forward mutation', :]
ctf4_clones = ctf4_clones.loc[ctf4_clones['mutation_direction'] == 'forward mutation', :]

In [9]:
# Import the list of yeast ORFs with size and description and merge it with the list of mutations
ORFs = pd.read_csv('ver_ORFs.tsv', sep='\t')
wt_evo1= pd.merge(wt_evo, ORFs, on='sgdid')
ctf4_pop1= pd.merge(ctf4_pop, ORFs, on='sgdid')
ctf4_clones1= pd.merge(ctf4_clones, ORFs, on='sgdid')
wt_evo2=wt_evo1[['sys_name','sgdid','gene_name','size','chr_num','position','sample_name','snp_indel','ref','read','fraction','tot_reads','mutation_type','origin','description']]
ctf4_pop2=ctf4_pop1[['sys_name','sgdid','gene_name','size','chr_num','position','sample_name','snp_indel','ref','read','fraction','tot_reads','mutation_type','origin','description']]
ctf4_clones2=ctf4_clones1[['sys_name','sgdid','gene_name','size','chr_num','position','sample_name','snp_indel','ref','read','fraction','tot_reads','mutation_type','origin','description']]

In [10]:
# Save the mutations to files before applying any other filter
wt_evo2.to_excel('output/mutations_wt_ver.xls')
ctf4_pop2.to_excel('output/mutations_ctf4_pop_ver.xls')
ctf4_clones2.to_excel('output/mutations_ctf4_clones_ver.xls')

In [11]:
# In case one gene has been mutated multiple times in the same line consider only one hit

# for wt
desired_rows = []
for name, cur_data in wt_evo2.groupby(['gene_name', 'sample_name']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
wt_s = pd.DataFrame(desired_rows)

# for ctf4 populations
desired_rows = []
for name, cur_data in ctf4_pop2.groupby(['gene_name', 'sample_name']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4pop_s = pd.DataFrame(desired_rows)

# for ctf4 clones

# remove the secon part of the sample_name and create a population column so that after we can count only one mutation per population
ctf4_clones2['population'] = ctf4_clones2.sample_name.str.split('_').str[0]

desired_rows = []
for name, cur_data in ctf4_clones2.groupby(['gene_name', 'population']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4clones_s = pd.DataFrame(desired_rows)

# Now, in the full list, remove the clones with the mutation in the same position otherwise they count as indivisual hits
desired_rows = []
for name, cur_data in ctf4_clones2.groupby(['population', 'position']): # This is a bit dirty, i'm assuming two different genes where not mutated in the same population at the same position, basically if two clones from the same populations have mutations in the same position, I'm assuming is the same gene and therefore I count it as 1 hit that spread.
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4clones_ss = pd.DataFrame(desired_rows)

In [12]:
#ANALYSIS OF MUTATIONS IN WT EVOLVED POPULATIONS (considering pop only)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(wt_s.sys_name) 
wt_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
wt_genes = wt_genes.rename(columns={'index':'sys_name', 0:'pop_hits'}) 
wt_genes=wt_genes.dropna()

#Merge the counter list with the original list
wt_list1= pd.merge(wt_s, wt_genes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in wt_list1.groupby(['sys_name', 'pop_hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
wt_s_h = pd.DataFrame(desired_rows)
wt_pop_hits=wt_s_h[['sys_name','sgdid','gene_name','size','pop_hits','origin','description']]
wt_pop_hits= wt_pop_hits.sort_values(by='pop_hits', ascending=0)
wt_pop_hits.to_excel('output/wt_pop_hits.xls')

#ANALYSIS OF MUTATIONS IN WT EVOLVED POPULATIONS (considering also clones)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(wt_evo2.sys_name) 
wt_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
wt_genes = wt_genes.rename(columns={'index':'sys_name', 0:'hits'}) 
wt_genes=wt_genes.dropna()

#Merge the counter list with the original list
wt_list2= pd.merge(wt_evo2, wt_genes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in wt_list2.groupby(['sys_name', 'hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
wt_s_h = pd.DataFrame(desired_rows)
wt_hits=wt_s_h[['sys_name','sgdid','gene_name','size','hits','origin','description']]
wt_hits= wt_hits.sort_values(by='hits', ascending=0)
wt_hits.to_excel('output/wt_hits.xls')

wt_summary = pd.merge(wt_pop_hits, wt_genes, on='sys_name', how='left')
wt_summary = wt_summary[['sys_name', 'sgdid', 'gene_name', 'size','pop_hits', 'hits', 'origin', 'description']]
wt_summary.to_excel('output/wt_hits_summary.xls')

In [13]:
#ANALYSIS OF MUTATIONS IN CTF4 POP (considering pop only)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(ctf4pop_s.sys_name) 
ctf4pop_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
ctf4pop_genes = ctf4pop_genes.rename(columns={'index':'sys_name', 0:'pop_hits'}) 
ctf4pop_genes=ctf4pop_genes.dropna()

#Merge the counter list with the original list
ctf4_list1= pd.merge(ctf4pop_s, ctf4pop_genes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in ctf4_list1.groupby(['sys_name', 'pop_hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4pop_s_h = pd.DataFrame(desired_rows)
ctf4pop_hits=ctf4pop_s_h[['sys_name','sgdid','gene_name','size','pop_hits','origin','description']]
ctf4pop_hits= ctf4pop_hits.sort_values(by='pop_hits', ascending=0)
ctf4pop_hits.to_excel('output/ctf4_pop_hits.xls')

#ANALYSIS OF MUTATIONS IN CTF4 POP (considering also clones)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(ctf4_pop2.sys_name) 
ctf4_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
ctf4_genes = ctf4_genes.rename(columns={'index':'sys_name', 0:'hits'}) 
ctf4_genes = ctf4_genes.dropna()

#Merge the counter list with the original list
ctf4_list2= pd.merge(ctf4_pop2, ctf4_genes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in ctf4_list2.groupby(['sys_name', 'hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4_h = pd.DataFrame(desired_rows)
ctf4_hits=ctf4_h[['sys_name','sgdid','gene_name','size','hits','origin','description']]
ctf4_hits= ctf4_hits.sort_values(by='hits', ascending=0)
ctf4_hits.to_excel('output/ctf4_hits.xls')

ctf4_hits = ctf4_hits.drop(['sgdid', 'gene_name','size', 'origin','description'], axis=1)
ctf4_summary = pd.merge(ctf4pop_hits, ctf4_hits, on='sys_name', how='right')
ctf4_summary = ctf4_summary[['sys_name', 'sgdid', 'gene_name', 'size','pop_hits', 'hits', 'origin', 'description']]
ctf4_summary.to_excel('output/ctf4_hits_summary.xls')

In [14]:
#ANALYSIS OF MUTATIONS IN CTF4 EVOLVED CLONES (only considering pop)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(ctf4clones_s.sys_name) 
ctf4clones_popgenes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
ctf4clones_popgenes = ctf4clones_popgenes.rename(columns={'index':'sys_name', 0:'pop_hits'}) 
ctf4clones_popgenes=ctf4clones_popgenes.dropna()

#Merge the counter list with the original list
ctf4clones_poplist= pd.merge(ctf4clones_s, ctf4clones_popgenes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in ctf4clones_poplist.groupby(['sys_name', 'pop_hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4clones_s_h = pd.DataFrame(desired_rows)
ctf4clones_pophits=ctf4clones_s_h[['sys_name','sgdid','gene_name','size','pop_hits','origin','description']]
ctf4clones_pophits= ctf4clones_pophits.sort_values(by='pop_hits', ascending=0)
ctf4clones_pophits.to_excel('output/ctf4clones_pop_hits.xls')

#ANALYSIS OF MUTATIONS IN CTF4 CLONES (considering also clones)

#Count the number of hits per genes and order them from highest to lowest
counter_pop=Counter(ctf4_clones2.sys_name) 
ctf4clones_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
ctf4clones_genes = ctf4clones_genes.rename(columns={'index':'sys_name', 0:'clones_hits'}) 
ctf4clones_genes = ctf4clones_genes.dropna()

#Merge the counter list with the original list
ctf4clones_list= pd.merge(ctf4_clones2, ctf4clones_genes, on='sys_name', how='left')

#Count the number of UNIQUE hits per genes and order them from highest to lowest
counter_pop=Counter(ctf4clones_ss.sys_name) 
ctf4unique_genes = pd.DataFrame.from_dict(counter_pop, orient='index').reset_index() 
ctf4unique_genes = ctf4unique_genes.rename(columns={'index':'sys_name', 0:'unique_hits'}) 
ctf4unique_genes = ctf4unique_genes.dropna()

#Merge the counter list with the original list
ctf4clones_list2= pd.merge(ctf4clones_list, ctf4unique_genes, on='sys_name', how='left')

desired_rows = []
for name, cur_data in ctf4clones_list2.groupby(['sys_name', 'clones_hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4_h = pd.DataFrame(desired_rows)
ctf4clones_hits=ctf4_h[['sys_name','sgdid','gene_name','size','clones_hits','origin','description']]
ctf4clones_hits= ctf4clones_hits.sort_values(by='clones_hits', ascending=0)
ctf4clones_hits.to_excel('output/ctf4clones_hits.xls')

desired_rows = []
for name, cur_data in ctf4clones_list2.groupby(['sys_name', 'unique_hits']):
    cur_row = cur_data.iloc[0]
    desired_rows.append(cur_row)
ctf4_h = pd.DataFrame(desired_rows)
ctf4unique_hits=ctf4_h[['sys_name','sgdid','gene_name','size','unique_hits','origin','description']]
ctf4unique_hits= ctf4unique_hits.sort_values(by='unique_hits', ascending=0)
ctf4unique_hits.to_excel('output/ctf4clones_unique_hits.xls')

ctf4clones_hits = ctf4clones_hits.drop(['sgdid', 'gene_name','size', 'origin','description'], axis=1)
ctf4unique_hits = ctf4unique_hits.drop(['sgdid', 'gene_name','size', 'origin','description'], axis=1)
ctf4clones_summary = pd.merge(ctf4clones_pophits, ctf4clones_hits, on='sys_name', how='right')
ctf4clones_summary2 = pd.merge(ctf4clones_summary, ctf4unique_hits, on='sys_name', how='right')
ctf4clones_summary2 = ctf4clones_summary2[['sys_name', 'sgdid', 'gene_name', 'size','pop_hits', 'unique_hits','clones_hits', 'origin', 'description']]
ctf4clones_summary2['unique_hits'] = ctf4clones_summary2['unique_hits'].apply(int)
ctf4clones_summary2.to_excel('output/ctf4clones_hits_summary.xls')

,sys_name,sgdid,gene_name,size,pop_hits,unique_hits,clones_hits,origin,description
0,YKL032C,S000001515,IXR1,1794,6,6,23,Intrastrand cross (X)-link Recognition,Transcriptional repressor that regulates hypox...
1,YGR095C,S000003327,RRP46,672,5,5,6,Ribosomal RNA Processing,Exosome non-catalytic core component; involved...
2,YDR217C,S000002625,RAD9,3930,4,7,16,RADiation sensitive,DNA damage-dependent checkpoint protein; requi...
3,YER011W,S000000813,TIR1,765,4,4,5,TIp1-Related,Cell wall mannoprotein; expression is downregu...
4,YLR264W,S000004254,RPS28B,204,4,4,10,Ribosomal Protein of the Small subunit,Protein component of the small (40S) ribosomal...
5,YDR227W,S000002635,SIR4,4077,3,6,7,Silent Information Regulator,SIR protein involved in assembly of silent chr...
6,YFL024C,S000001870,EPL1,2499,3,4,9,Enhancer of Polycomb Like,"Subunit of NuA4, an essential histone H4/H2A a..."
7,YLR207W,S000004197,HRD3,2502,2,2,2,HMG-coA Reductase Degradation,ER membrane protein that plays a central role ...
8,YLR442C,S000004434,SIR3,2937,2,5,5,Silent Information Regulator,"Silencing protein; interacts with Sir2p, Sir4p..."
9,YLR445W,S000004437,S000004437,649,2,4,4,Grand Meiotic recombination Cluster,Protein involved in meiotic crossing over; com...


In [15]:
# ADAPTIVE MUTATIONS IN WT

#get the total mutations obtained at the end of the experiment (counting pop_hits I only consider unique mutations in pop)
n=wt_summary['pop_hits'].sum()
#get the size of all the genes mutated + 500bp for promoters/terminators
target_size = (wt_summary['size'].sum())+(len(wt_summary)*500)
#get the size of all the genes in the genome + 500bp for promoters/terminators
genome_size = (ORFs['size'].sum())+(len(ORFs)*500)
#calculate the rate of mutations obtained at the end of the experiment
random_rate=float(n)/genome_size

#for each gene hit, calculate a mutation rate
from scipy.special import gammaincc
from scipy.special import factorial
# Probability of having >=n number of hits
wt_summary['P']= 1-gammaincc(wt_summary['pop_hits']+1, random_rate*(wt_summary['size']+500))
# Probability of having =n number of hits
wt_summary['Pe']= (random_rate*(wt_summary['size']+500))**wt_summary['pop_hits']*exp(-random_rate*(wt_summary['size']+500))/factorial(wt_summary['pop_hits'])

#Benjamini-Hochberg correction
positive_bh=wt_summary.sort_values(by='P')
positive_bh=positive_bh.reset_index()
positive_bh2=positive_bh.loc[positive_bh['P'] < (0.01*(positive_bh.index+1)/len(ORFs)), :]
#positive_bh=positive_bh.head(len(positive_bh2))
positive_bh.to_excel('output/wt_hits_summary_Pvalue_all.xls')
positive_bh2.to_excel('output/wt_hits_summary_Benjamini-Hochberg.xls')

#Bonferroni correction
positive_bo = wt_summary.loc[wt_summary['P'] < (0.05/len(ORFs)), :]
#positive_bo=positive_bo.reset_index()
positive_bo.to_excel('output/wt_hits_summary_bonferroni.xls')

In [16]:
# removal of mutations selected in wt from the mutations found in the ctf4 evo experiment

ctf4_summary_filtered = ctf4_summary[~ctf4_summary.sys_name.isin(positive_bo.sys_name)]
ctf4clones_summary_filtered = ctf4clones_summary[~ctf4clones_summary.sys_name.isin(positive_bo.sys_name)]
ctf4_summary_filtered.to_excel('output/ctf4_summary_filtered.xls')
ctf4clones_summary_filtered.to_excel('output/ctf4clones_summary_filtered.xls')

In [17]:
# ADAPTIVE MUTATIONS IN CTF4 EVO POP

#get the total mutations obtained at the end of the experiment (counting pop_hits I only consider unique mutations in pop)
n=ctf4_summary['pop_hits'].sum()

#get the size of all the genes in the genome + 500bp for promoters/terminators
genome_size = (ORFs['size'].sum())+(len(ORFs)*500)

#calculate the rate of mutations obtained at the end of the experiment
random_rate=float(n)/genome_size

#for each gene hit, calculate a mutation rate
from scipy.special import gammaincc
from scipy.special import factorial
ctf4_summary['P']= 1-gammaincc(ctf4_summary['pop_hits']+1, random_rate*(ctf4_summary['size']+500))
ctf4_summary['Pe']= (random_rate*(ctf4_summary['size']+500))**ctf4_summary['pop_hits']*exp(-random_rate*(ctf4_summary['size']+500))/factorial(ctf4_summary['pop_hits'])

#Benjamini-Hochberg correction
positive_bh=ctf4_summary.sort_values(by='P')
positive_bh=positive_bh.reset_index()
positive_bh2=positive_bh.loc[positive_bh['P'] < (0.05*(positive_bh.index+1)/len(ORFs)), :]
#positive_bh=positive_bh.head(len(positive_bh2))
positive_bh.to_excel('output/ctf4_summary_Benjamini-Hochberg.xls')

#Bonferroni correction
positive_bo = ctf4_summary.loc[ctf4_summary['P'] < (0.05/len(ORFs)), :]
#positive_bo=positive_bo.reset_index()
positive_bo.to_excel('output/ctf4_summary_bonferroni.xls')

In [18]:
# ADAPTIVE MUTATIONS IN CTF4 EVO  CLONES

#get the total mutations obtained at the end of the experiment (counting pop_hits I only consider unique mutations in pop)
n=ctf4clones_summary['pop_hits'].sum()

#get the size of all the genes in the genome + 500bp for promoters/terminators
genome_size = (ORFs['size'].sum())+(len(ORFs)*500)

#calculate the rate of mutations obtained at the end of the experiment
random_rate=float(n)/genome_size

#for each gene hit, calculate a mutation rate
from scipy.special import gammaincc
from scipy.special import factorial
ctf4clones_summary['P']= 1-gammaincc(ctf4clones_summary['pop_hits']+1, random_rate*(ctf4clones_summary['size']+500))
ctf4clones_summary['Pe']= (random_rate*(ctf4clones_summary['size']+500))**ctf4clones_summary['pop_hits']*exp(-random_rate*(ctf4clones_summary['size']+500))/factorial(ctf4clones_summary['pop_hits'])

#Benjamini-Hochberg correction
positive_bh=ctf4clones_summary.sort_values(by='P')
positive_bh=positive_bh.reset_index()
positive_bh2=positive_bh.loc[positive_bh['P'] < (0.05*(positive_bh.index+1)/len(ORFs)), :]
#positive_bh=positive_bh.head(len(positive_bh2))
positive_bh.to_excel('output/ctf4_hits_summary_Pvalue_all.xls')
positive_bh2.to_excel('output/ctf4_hits_summary_Benjamini-Hochberg.xls')

#Bonferroni correction
positive_bo = ctf4clones_summary.loc[ctf4clones_summary['P'] < (0.05/len(ORFs)), :]
#positive_bo=positive_bo.reset_index()
positive_bo.to_excel('output/ctf4clones_summary_bonferroni.xls')